In [5]:
import numpy as np

### Processing timeSeries using reccurent neural networks

In [6]:
def generate_time_series(batch_size, n_steps):
    
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) #
    #wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # +
    #wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    # +noise
    return series[..., np.newaxis].astype(np.float32)

### Creating a training and validation set

In [7]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

x_train, y_train = series[:70000:n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]


### Getting a baseline metric

In [ ]:
y_pred = X_valid[:, -1]

np.mean(keras.losses.mean_squared_error(y_valid, y_pred))


### using the previous time step as out put calculate mse

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [50,1]),
    keras.layers.Dense(1)
])

In [9]:
#TO DO COMPILE MODEL AND THEN FIT IT
np.max(X_test)


0.7481007

### Creating a simPLE dEEP RNN


In [ ]:
model = keras.models.sequential([
    keras.layers.SimpleRNN(20, return_sequences = True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences = True), 
    keras.layers.SimpleRNN(1)# if you only care about last output  only
])

### For faster implementation

In [ ]:
model = keras.models.sequential([
    keras.layers.SimpleRNN(20, return_sequences = True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences = True), 
    keras.layers.Dense(1)# if you only care about last output  only
])

### Predicting several time Steps ahead

> Method 1: predicting on time step ahead and then appending that prediction to data used and then predictin it again

In [ ]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
Y_pred = X[:, n_steps:]

> Method 2: training an RNN to predict all 10 next values at once
    
    To use a sequence to vector model we will have to change how we create the the
    training and testing set

In [13]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:,
0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [10]:
### The resulting network will look like

In [ ]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,
1]),
keras.layers.SimpleRNN(20),
keras.layers.Dense(10)
])

### using sequence to sequence model

- The advantage of this is that the rnn will contain loss terms from all the time steps, not just output time step
- This means many more error gradients flowing in the network
- This both stabilizes and speed up training

In [14]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10Dvectors
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps,0]

In [15]:
Y.shape

(10000, 50, 10)

In [ ]:
#(take in 3 dimensional matrixx)

In [ ]:
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

### The corresponding model will look like this

In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,
    1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])


# use time distributed to make a dense layer acccpet a sequence as inputs 
# same as Dense 10

Since all the outputs are needed during training, but only the last time
step is useful for prediction and for evaluation ,we will rely on MSE over all
outputs for training, we will use a custom metric for evaluation

In [ ]:
def last_time_step_mse(y_true, y_pred):
    return keras.metrics.mean_squared_error(y_true[:,-1], y_pred[:,-1])

opitimizer = keras.optimizers.Adam(lr = 0.01)
model.compile(loss = "mse", optimizer = optimizer, metrics = [last_time_step_mse])

We get a validation MSE of about 0.006, which is 25% better than the
previous model. You can combine this approach with the first one: just
predict the next 10 values using this RNN, then concatenate these values to
the input time series and use the model again to predict the next 10 values,
and repeat the process as many times as needed. With this approach, you
can generate arbitrarily long sequences. It may not be very accurate for
long-term predictions, but it may be just fine if your goal is to generate
original music or text, as we will see in

In [ ]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
    super().__init__(**kwargs)
    self.state_size = units
    self.output_size = units
    self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
    activation=None)
    self.layer_norm = keras.layers.LayerNormalization()
    self.activation = keras.activations.get(activation)
    def call(self, inputs, states):
    outputs, new_states = self.simple_rnn_cell(inputs, states)
    norm_outputs = self.activation(self.layer_norm(outputs))
    return norm_outputs, [norm_outputs]

In [ ]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
    input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
    ])

### Dealing with short term memory (LSTM cells)

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
    ])

### Dealing with the fairly short term memort of the LSTM

a 1D convolutional
layer slides several kernels across a sequence, producing a 1D feature map
per kernel. Each kernel will learn to detect a single very short sequential
pattern (no longer than the kernel size). If you use 10 kernels, then the
layer’s output will be composed of 10 1-dimensional sequences (all of the
same length), or equivalently you can view this output as a single 10-
dimensional sequence. This means that you can build a neural network
composed of a mix of recurrent layers and 1D convolutional layers (or
even 1D pooling layers). If you use a 1D convolutional layer with a stride
of 1 and "same" padding, then the output sequence will have the same
length as the input sequence. But if you use "valid" padding or a stride
greater than 1, then the output sequence will be shorter than the input
sequence,

### Example model

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2,
    padding="valid",
    input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])


# the first layer keeps important information while also shortenniing the sequence making the lstm keep a longer memory

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
validation_data=(X_valid, Y_valid[:, 3::2]))

### WAVENET

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2,
    padding="causal",
    activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))


model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
validation_data=(X_valid, Y_valid))